In [1]:
import os
import time
import copy
import glob
import cv2
import shutil

import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [2]:
# 이미지 데이터 전처리 방법 정의

data_path = 'data/catanddog/train'

trsf = transforms.Compose(
    [
        transforms.Resize([256, 256]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ]
)
# torchvision.transforms -> 이미지 데이터를 변환하여 모델의 입력으로 사용할 수 있게 변환
# Resize : 이미지의 크기를 재조정
# RandomResizedCrop : 랜덤한 크기 및 비율로 자름
# RandomHorizontalFlip : 이미지를 랜덤하게 수평으로 뒤집음
# ToTensor : 이미지 데이터를 텐서로 변환

train_dataset = torchvision.datasets.ImageFolder(
    data_path, transform= trsf
)
# ImageFolder -> 데이터로더가 데이터를 불러올 경로와 방법

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size= 32,
    num_workers= 8,
    shuffle= True
)

print(len(train_dataset))

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: 'data/catanddog/train'

In [3]:
# 사전 훈련된 모델
resnet18 = models.resnet50(pretrained= True)

# 파라미터 학습 유무 지정
def set_parameter_requires_grad(model, feature_extracting= True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

set_parameter_requires_grad(resnet18)

# 완전연결층 추가
resnet18.fc = nn.Linear(512, 2)

# 파라미터 값 확인
for name, param in resnet18.named_parameters():
    if param.requires_grad:
        print(name, param.data)

c:\Users\USER\anaconda3\envs\env02\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\anaconda3\envs\env02\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\USER/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 66.8MB/s]

fc.weight tensor([[ 0.0343, -0.0108,  0.0328,  ..., -0.0090,  0.0408,  0.0252],
        [ 0.0088, -0.0399,  0.0402,  ...,  0.0282, -0.0075, -0.0299]])
fc.bias tensor([-0.0110, -0.0388])


In [4]:
# 모델 객체 생성, 손실 함수 정의

model = models.resnet50(pretrained= True)

for param in model.parameters(): # 합성곱층 가중치 고정
    param.required_grad = False

model.fc = torch.nn.Linear(512,2)
for param in model.fc.parameters(): # 완전연결층은 학습
    param.requires_grad = True

opt = torch.optim.Adam(model.fc.parameters())
cost = torch.nn.CrossEntropyLoss()

print(model)

c:\Users\USER\anaconda3\envs\env02\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\USER\anaconda3\envs\env02\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
p = model.parameters()
print(type(model.parameters()))
print(p)

<class 'generator'>
<generator object Module.parameters at 0x0000023AB504C580>
